# IO


## Export Tiff Image with Meta Data

In [ ]:
from scitiff.data import hyperstack_example
import scipp as sc

sample_img = hyperstack_example()
sample_img

In [ ]:
config = {"grid": True, "aspect": True}
img = sample_img.mean('c').mean('z')
(img['t', 0].plot(title='T=0', **config) + img['t', 1].plot(title='T=1', **config)) / (
    img['t', 2].plot(title='T=2', **config) + img['t', 3].plot(title='T=3', **config)
)

In [ ]:
from scitiff.io import save_scitiff

save_scitiff(sample_img, 'sample_img.tiff')

## Load TIFF Image with Meta Data

In [ ]:
from scitiff.io import load_scitiff

loaded_scitiff: sc.DataGroup[sc.DataArray] = load_scitiff('sample_img.tiff')
loaded_scitiff

In [ ]:
loaded_scitiff['image']

In [ ]:
config = {"grid": True, "aspect": True}
loaded_img = loaded_scitiff['image'].mean('c').mean('z')
(
    loaded_img['t', 0].plot(title='T=0', **config)
    + loaded_img['t', 1].plot(title='T=1', **config)
) / (
    loaded_img['t', 2].plot(title='T=2', **config)
    + loaded_img['t', 3].plot(title='T=3', **config)
)

## Load Non-Scitiff Images

Sometimes you might want to load tiff images that are not compatible with scitiff.
i.e. no metadata, incompatible dtype, or broken metadata with wrong fields or values.

``load_scitiff`` will try to load metadata and handle incompatible profiles, but if it is not possible it will load the tiff image as it is and wrap it into scipp data structure.

.. note::
    ``load_scitiff` will throws warning if it fails to load the file as expected way
    so please pay attention to the warning messages.

In [ ]:
import tifffile as tf

tf.imwrite('no-meta.tiff', data=sample_img.values)
no_meta_loaded_image = load_scitiff('no-meta.tiff')['image']
display(no_meta_loaded_image)
no_meta_loaded_image_reduced = no_meta_loaded_image.sum('dim_0').sum('dim_2')
(
    no_meta_loaded_image_reduced['dim_1', 0].plot(title='dim_1=0', **config)
    + no_meta_loaded_image_reduced['dim_1', 1].plot(title='dim_1=1', **config)
) / (
    no_meta_loaded_image_reduced['dim_1', 2].plot(title='dim_1=2', **config)
    + no_meta_loaded_image_reduced['dim_1', 3].plot(title='dim_1=3', **config)
)

.. tip::
    You can rename the dimensions and use it just like other scitiff images.
    Do it only if you know which `dim_{i}` corresponds to which of ('t', 'z', 'c', 'y', 'x').

In [ ]:
renamed_image = no_meta_loaded_image.rename_dims({
    "dim_0": "c", "dim_1": "t", "dim_2": "z", "dim_3": "y", "dim_4": "x"
})
display(renamed_image)
renamed_image_reduced = renamed_image.sum('z').sum('c')
(
    renamed_image_reduced['t', 0].plot(title='T=0', **config)
    + renamed_image_reduced['t', 1].plot(title='T=1', **config)
) / (
    renamed_image_reduced['t', 2].plot(title='T=2', **config)
    + renamed_image_reduced['t', 3].plot(title='T=3', **config)
)